In [1]:
import os

In [ ]:
%pwd

In [3]:
os.chdir("../")

In [ ]:
%pwd

In [5]:
import pandas as pd

In [6]:
data = pd.read_csv("artifacts/data_ingestion/Housing.csv")
data.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   price             545 non-null    int64 
 1   area              545 non-null    int64 
 2   bedrooms          545 non-null    int64 
 3   bathrooms         545 non-null    int64 
 4   stories           545 non-null    int64 
 5   mainroad          545 non-null    object
 6   guestroom         545 non-null    object
 7   basement          545 non-null    object
 8   hotwaterheating   545 non-null    object
 9   airconditioning   545 non-null    object
 10  parking           545 non-null    int64 
 11  prefarea          545 non-null    object
 12  furnishingstatus  545 non-null    object
dtypes: int64(6), object(7)
memory usage: 55.5+ KB


In [8]:
data.isnull().sum()

price               0
area                0
bedrooms            0
bathrooms           0
stories             0
mainroad            0
guestroom           0
basement            0
hotwaterheating     0
airconditioning     0
parking             0
prefarea            0
furnishingstatus    0
dtype: int64

In [9]:
data.shape

(545, 13)

In [10]:
data.columns

Index(['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'mainroad',
       'guestroom', 'basement', 'hotwaterheating', 'airconditioning',
       'parking', 'prefarea', 'furnishingstatus'],
      dtype='object')

----------------------------------- entity ------------------------------------------

In [13]:
from dataclasses import dataclass
from pathlib import Path

# Definir una clase de datos inmutable que almacena 
# la configuración para la validación de datos
@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

----------------------------------- configuration manager ------------------------------------------

In [14]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [15]:
class ConfigurationManager:
    # Definir el método constructor que recibe las rutas 
    # de los archivos de configuración, parámetros y esquema
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        # Leer los archivos YAML y asignarlos a los atributos de la instancia
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        # Crear el directorio raíz para los artefactos del proyecto
        create_directories([self.config.artifacts_root])

# ------------------------------------------------------------------------------------------------
    # Definir un método que devuelve la configuración para la validacion de datos
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS # obtener las columnas desde el archivo yaml
        # Crear el directorio raíz para la validación de datos
        create_directories([config.root_dir])
        # Crear un objeto de la clase DataValidationConfig con los parámetros de la configuración
        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

----------------------------------- components ------------------------------------------

* ----------------------------------- data_validation ------------------------------------------

In [16]:
import os
from mlProject import logger

In [17]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    # Definir un método que valida los datos obtenidos después de la ingestión de datos
    def validate_all_columns(self)-> bool:
        try:
            validation_status = None
            # cargar el dataset de la ruta en el cual se desenpaquetó
            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns) # obtener los nombre de las columnas
            # recuperar el nombre de las columnas(features) 
            # definidos en nuestro schema
            all_schema = self.config.all_schema.keys()

            # iteramos sobre todas las columnas
            for col in all_cols:
                # verificamos que col no se encuentre dentro de lo que se ha definido en el schema
                if col not in all_schema:
                    # sino se encuentra, asignamos False
                    validation_status = False 
                    # luego abrimos el archivo y guardamos el valor de "validation_status"
                    with open(self.config.STATUS_FILE, 'w') as f: 
                        f.write(f"Validation status: {validation_status}")
                else:
                    # si se encuentra, asignamos True
                    validation_status = True
                    # luego abrimos el archivo y guardamos el valor de "validation_status"
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e


----------------------------------- pipeline ------------------------------------------

* ----------------------------------- stage_02_data_validation_pipeline ------------------------------------------

In [18]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2023-08-19 09:39:00,297: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-19 09:39:00,306: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-19 09:39:00,331: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-08-19 09:39:00,336: INFO: common: created directory at: artifacts]
[2023-08-19 09:39:00,340: INFO: common: created directory at: artifacts/data_validation]
